# Day 16: Packet Decoder

As you leave the cave and reach open waters, you receive a transmission from the Elves back on the ship.

The transmission was sent using the Buoyancy Interchange Transmission System (BITS), a method of packing numeric expressions into a binary sequence. Your submarine's computer has saved the transmission in hexadecimal (your puzzle input).

## Input

The BITS transmission contains a single packet at its outermost layer which itself contains many other packets. The hexadecimal representation of this packet might encode a few extra 0 bits at the end; these are not part of the transmission and should be ignored.

Every packet begins with a standard header: the first three bits encode the packet version, and the next three bits encode the packet type ID. These two values are numbers; all numbers encoded in any packet are represented as binary with the most significant bit first.

Packets with type ID 4 represent a literal value. Literal value packets encode a single binary number. To do this, the binary number is padded with leading zeroes until its length is a multiple of four bits, and then it is broken into groups of four bits. Each group is prefixed by a 1 bit except the last group, which is prefixed by a 0 bit. These groups of five bits immediately follow the packet header.

In [ ]:
class Packet {
    public byte Version { get; private set; }
    public byte TypeId { get; private set; }

    public long Literal { get; private set; }
    public Packet[] SubPackets { get; private set; } = Array.Empty<Packet>();

    public override string ToString()
        => $"{Version:d3}/{TypeId:d3} " +
            (TypeId == 4 ? Literal.ToString("d8") :
            "[" + string.Join(",", SubPackets.Select(s => s.ToString())) + "]");

    static public (Packet, string) Parse(string input) {
        var version = Convert.ToByte(input[0..3], 2);
        var typeId = Convert.ToByte(input[3..6], 2);

        var packet = new Packet { Version = version, TypeId = typeId };
        var remain = input[6..];

        if (packet.TypeId == 4) {
            var last = '1';
            var literal = "";
            while (last == '1') {
                last = remain[0];
                literal = literal + remain[1..5];
                remain = remain[5..];
            }
            packet.Literal = Convert.ToInt64(literal, 2);
        } else {
            var subPackets = new List<Packet>();
            if (remain[0] == '0') {
                var last = Convert.ToInt16(remain[1..16], 2) + 16;
                var inside = remain[16..last];
                remain = remain[last..];
                while (inside.Length > 5) {
                    (var sub, inside) = Packet.Parse(inside);
                    subPackets.Add(sub);
                }
            } else {
                var packets = Convert.ToInt16(remain[1..12], 2);
                remain = remain[12..];
                for (var i = 0; i < packets; i++) {
                    (var sub, remain) = Packet.Parse(remain);
                    subPackets.Add(sub);
                }
            }
            packet.SubPackets = subPackets.ToArray();
        }

        return (packet, remain);
    }
}

In [ ]:
using System.IO;
// var input = File.ReadAllLines(@"day-16.sample");
var input = File.ReadAllLines(@"day-16.input");

static string FromHex(string hex)
    => string.Join("", hex.ToCharArray().Chunk(2)
        .Select(l => new String(l)).Select(h => Convert.ToByte(h, 16))
        .Select(c => "0000000" + Convert.ToString(c, 2))
        .Select(s => s[^8..^0]));

var bits = input.Select(FromHex).ToArray();

var packets = bits.Select(Packet.Parse).ToArray();

// foreach (var p in packets) Console.WriteLine(p);

packets.Length

1

## Part 1

Decode the structure of your hexadecimal-encoded BITS transmission; what do you get if you add up the version numbers in all packets?

In [ ]:
static long SumVersions(Packet p)
    => p.Version + p.SubPackets.Sum(s => SumVersions(s));

packets.Select(s => SumVersions(s.Item1))

index,value
0,971


## Part 2

Now that you have the structure of your transmission decoded, you can calculate the value of the expression it represents.

Literal values (type ID 4) represent a single number as described above. The remaining type IDs are more interesting:

- Packets with type ID 0 are sum packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 1 are product packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 2 are minimum packets - their value is the minimum of the values of their sub-packets.
- Packets with type ID 3 are maximum packets - their value is the maximum of the values of their sub-packets.
- Packets with type ID 5 are greater than packets - their value is 1 if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 6 are less than packets - their value is 1 if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 7 are equal to packets - their value is 1 if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.

Using these rules, you can now work out the value of the outermost packet in your BITS transmission.

In [ ]:
long Evaluate(Packet p) {
    if (p.TypeId == 4) return p.Literal;

    var subPackets = p.SubPackets.Select(s => Evaluate(s)).ToArray();

    return p.TypeId switch {
        0 => subPackets.Sum(),
        1 => subPackets.Aggregate(1L, (a, s) => a * s),
        2 => subPackets.Min(),
        3 => subPackets.Max(),

        5 => subPackets[0] > subPackets[1] ? 1 : 0,
        6 => subPackets[0] < subPackets[1] ? 1 : 0,
        7 => subPackets[0] == subPackets[1] ? 1 : 0,
    };
}

packets.Select(s => (Evaluate(s.Item1), s.ToString()))

index,Item1,Item2
0,831996589851,"(004/000 [002/001 [004/006 [000/004 00000079,004/004 00000018],005/004 00000130],006/001 [006/006 [002/004 00000017,006/004 00003739],001/004 00026181],007/003 [007/004 3381457610,003/004 00000013],004/000 [002/004 00368803,001/004 00000046,007/004 30827667693,003/004 172659535],003/001 [006/006 [003/004 00030901,006/004 00000011],007/004 229877055],003/001 [007/004 00000796,007/007 [007/000 [002/004 00000009,000/004 00000015,003/004 00000007],007/000 [007/004 00000011,006/004 00000004,007/004 00000006]]],003/001 [001/005 [007/004 00032638,000/004 00032638],000/004 00003874],001/004 00043547,006/004 00009965,004/002 [007/004 00482015,002/004 00003103,006/004 00654457,002/004 00057514,000/004 00000006],005/004 00003139,001/003 [004/004 00003605],003/001 [006/006 [007/004 102088296,004/004 102088296],007/004 00003835],006/004 00001952,000/000 [000/004 00000010,006/004 00054567,003/004 14720449],003/001 [006/006 [000/000 [006/004 00000014,001/004 00000014,003/004 00000010],007/000 [005/004 00000010,007/004 00000015,001/004 00000007]],001/004 02194231],007/004 00000005,004/001 [000/004 00000223,002/007 [000/004 00003909,001/004 00001470]],000/004 00000003,005/003 [001/004 00000201,002/004 00035420,002/004 00573540],004/004 45652672,002/001 [001/003 [005/003 [006/000 [007/000 [000/000 [007/002 [001/003 [004/001 [001/000 [003/002 [004/001 [004/002 [005/000 [001/003 [005/003 [001/001 [007/002 [002/001 [007/001 [003/004 00000015]]]]]]]]]]]]]]]]]]]],005/001 [001/005 [006/000 [004/004 00000015,004/004 00000005,004/004 00000014],005/000 [003/004 00000011,002/004 00000007,006/004 00000003]],000/004 00000070],001/002 [000/004 00002044,005/004 00003881],006/004 00000004,000/001 [002/004 00000158,005/005 [000/004 00010409,000/004 15363456644]],004/004 00000198,004/001 [005/004 00000142,006/007 [000/004 00005186,000/004 00005186]],000/000 [004/004 00047679,005/004 00000084,005/004 251511348,001/004 232790479076,007/004 00000002],003/000 [000/004 00001395],005/001 [001/004 00000038],005/001 [005/004 00000111,001/004 00000149,001/004 00000037,000/004 00000064,007/004 00000250],007/001 [001/004 00000247,005/004 00000042],001/001 [003/005 [004/004 00039675,004/004 00006739],001/004 00002732],003/001 [007/000 [005/004 00000008,002/004 00000004,001/004 00000014],005/000 [006/004 00000014,001/004 00000005,000/004 00000015],002/000 [003/004 00000013,004/004 00000015,003/004 00000003]],007/004 00002108,007/002 [001/004 00018914,005/004 00000012,001/004 00041461,001/004 00000052],000/002 [006/004 00704684,001/004 00000122,005/004 00005122],000/001 [003/004 00000214,002/006 [003/004 00778202,006/004 00778202]],006/001 [002/006 [006/004 06611022,004/004 08088420],002/004 00001963],006/001 [005/004 00438324,002/005 [003/004 00002033,007/004 00002033]],006/001 [007/005 [007/004 1672195128,006/004 00019983],003/004 00004357],004/003 [000/004 00000008,005/004 00000104,000/004 00000001,004/004 551251961610],000/001 [004/004 00003793,006/007 [001/004 00000240,006/004 00018572]],007/003 [002/004 00000871,004/004 3257895716,001/004 00000066,003/004 00000007,005/004 2738196270],005/000 [000/001 [003/004 00000009,005/004 00000011,000/004 00000008],003/001 [007/004 00000011,005/004 00000004,005/004 00000015],001/001 [006/004 00000014,007/004 00000005,006/004 00000008]],003/000 [003/004 00003841,004/004 00000009],007/001 [005/006 [005/000 [006/004 00000008,004/004 00000009,003/004 00000013],001/000 [003/004 00000007,007/004 00000006,004/004 00000005]],006/004 00000231],004/001 [003/004 00000064,007/004 00000175,002/004 00000157],005/001 [001/004 00000147,006/004 00000047,000/004 00000046,001/004 00000222],000/001 [000/004 00000220,006/005 [001/004 00000101,006/004 00063580]],002/002 [005/004 137990907],001/001 [005/004 00000153,005/005 [006/000 [003/004 00000012,005/004 00000011,006/004 00000014],007/000 [002/004 00000014,005/004 00000009,005/004 00000004]]]], 000000)"
